In [1]:
import numpy as np
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from pathlib import PurePath
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import keras
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input, BatchNormalization
from keras.optimizers import SGD, Adam
from keras import regularizers
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Add


In [2]:
suites_images_train = np.load('../getDataSet/dataset/images_train.npy')
suites_labels_train = np.load('../getDataSet/dataset/labels_train.npy')[:,1]
suites_images_test = np.load('../getDataSet/dataset/images_test.npy')
suites_labels_test = np.load('../getDataSet/dataset/labels_test.npy')[:,1]
print(Counter(suites_labels_train))
print(Counter(suites_labels_test))

le = LabelEncoder()
le.fit(suites_labels_train)
suites_labels_train = le.transform(suites_labels_train)
suites_labels_test = le.transform(suites_labels_test)

Counter({'d': 1208, 's': 1202, 'h': 1198, 'c': 1176})
Counter({'c': 318, 'h': 298, 'd': 291, 's': 289})


In [3]:
# Create the Sequential model
suitesmodel = Sequential()

# Add input layer
suitesmodel.add(Input(shape=(70, 70, 1)))

# Add first convolutional layer with batch normalization, pooling, and dropout
suitesmodel.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
suitesmodel.add(BatchNormalization())
suitesmodel.add(MaxPooling2D((2, 2)))
suitesmodel.add(Dropout(0.3))

# Add second convolutional layer with batch normalization, pooling, and dropout
suitesmodel.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
suitesmodel.add(BatchNormalization())
suitesmodel.add(MaxPooling2D((2, 2)))
suitesmodel.add(Dropout(0.3))

# Add third convolutional layer with batch normalization, pooling, and dropout
suitesmodel.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
suitesmodel.add(BatchNormalization())
suitesmodel.add(MaxPooling2D((2, 2)))
suitesmodel.add(Dropout(0.3))

# Add fourth convolutional layer with batch normalization, pooling, and dropout
suitesmodel.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
suitesmodel.add(BatchNormalization())
suitesmodel.add(MaxPooling2D((2, 2)))
suitesmodel.add(Dropout(0.4))

# Flatten the output and add dense layers
suitesmodel.add(Flatten())
suitesmodel.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
suitesmodel.add(Dense(4, activation='softmax'))

# Compile the model
opt = keras.optimizers.Adam(learning_rate=0.001)
suitesmodel.compile(optimizer=opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

# Print the model summary
suitesmodel.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 70, 70, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 70, 70, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 35, 35, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 35, 35, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 35, 35, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 35, 35, 64)       

In [4]:
suitesmodel.fit(suites_images_train, suites_labels_train,epochs=30, validation_data=(suites_images_test, suites_labels_test),batch_size=64)

Epoch 1/30


75/75 [==============================] - 17s 198ms/step - loss: 1.8847 - accuracy: 0.2952 - val_loss: 3.0175 - val_accuracy: 0.2492
Epoch 2/30
75/75 [==============================] - 15s 194ms/step - loss: 1.3653 - accuracy: 0.4480 - val_loss: 3.1806 - val_accuracy: 0.2500
Epoch 3/30
75/75 [==============================] - 14s 187ms/step - loss: 0.9805 - accuracy: 0.6135 - val_loss: 4.8347 - val_accuracy: 0.2659
Epoch 4/30
75/75 [==============================] - 14s 187ms/step - loss: 0.7742 - accuracy: 0.7172 - val_loss: 11.0929 - val_accuracy: 0.2659
Epoch 5/30
75/75 [==============================] - 14s 188ms/step - loss: 0.6427 - accuracy: 0.7751 - val_loss: 15.4427 - val_accuracy: 0.2659
Epoch 6/30
75/75 [==============================] - 14s 187ms/step - loss: 0.5461 - accuracy: 0.8223 - val_loss: 16.6100 - val_accuracy: 0.2667
Epoch 7/30
75/75 [==============================] - 14s 193ms/step - loss: 0.4722 - accuracy: 0.8641 - val_loss: 10.1322 - val_accuracy: 

In [6]:
suitesmodel.save("./models/SuitesCNN.keras")
np.save('./encoders/SuitesEncoder.npy', le.classes_)